1. Find which block is in developed region.

In [1]:
from osgeo import gdal
import numpy as np

data_path = '0_data/'


blocks = gdal.Open(data_path + 'block_raster.tif')
blocks_band = blocks.GetRasterBand(1)
blocks_data = blocks.ReadAsArray()
row_length = blocks.RasterYSize
col_length = blocks.RasterXSize
tif_geotrans = blocks.GetGeoTransform()
tif_proj = blocks.GetProjection()


regions = gdal.Open(data_path + 'obs_regions.tif')
regions_band = regions.GetRasterBand(1)
regions_data = regions.ReadAsArray()


block_region = []
block_max_id = blocks_data.max()
for i in range( block_max_id + 1 ):
    block_region.append([])

for row in range(row_length):
    for col in range(col_length):
        block_id = blocks_data[row,col]
        if block_id > 0:
            region = regions_data[row, col]
            block_region[block_id].append(region)

2. Find which record is not in developed block.

In [3]:
import csv

obsall_csv = csv.reader(open(data_path + "2017stall.csv"))
obsall_data = np.array(list(obsall_csv))

developing_index = []
for record_i in range(1, len(obsall_data)):
    block_id = int(obsall_data[record_i, 1])
    if len(block_region[block_id]) > 0:
        block_isregion = int(block_region[block_id][0])
        if block_isregion == 0:
            developing_index.append(record_i)

In [5]:
developing_data_filename = data_path + "developing_2017stall.csv"
with open (developing_data_filename, "w", newline = '') as developing_data:
    developing_data_csv = csv.writer(developing_data)
    developing_data_csv.writerow(obsall_data[0,:])
    for i in developing_index:
        developing_data_csv.writerow(obsall_data[i,:])